In [45]:
%reload_ext autoreload
%autoreload 2
import os, sys
sys.path.append('../')
import logging
import random
import numpy as np, pandas as pd
import config
from utilities import *
from _rulesbuilding import *
from IPython.display import display, HTML

In [46]:
# Load the ruleset
rules = pd.read_csv(config.inputs['rules']['fullpath'])
pd.options.mode.chained_assignment = None  # default='warn'

In [48]:
"""
Requirement
Add the requirements to the regex builder
1. S (start): if the pattern in ​text_match​ 
column is a prefix of the description string, 
a match is found for the corresponding service 
(both ID and name are included in the table)
2. A (anywhere): similar to above, except 
that the pattern doesn’t have to be in the 
beginning of the description
3. R (regular expression): use ​text_match​ 
as a regular regular expression
"""

_rules = rules
_rules = _rules[(_rules['text_match'].notna())]
_rules['text_match'].fillna('.*', inplace = True)

_rules['text_exclude'].fillna('a^', inplace = True)
_rules['text_exclude'] = '(?!.*(' + _rules['text_exclude'] + ').*).*$'

prefix = np.where(_rules['matching'] == 'A', '^(?=.*(', '^(?=(') 
_rules['text_match'] = prefix + _rules['text_match'] + ').*)' + _rules['text_exclude']

_rules = _rules.groupby('service_id')['text_match'].apply(
            lambda t: '|'.join([str(i) for i in t])
        ).astype(str)


regexes_dict = {}
inv_map = {v: k for k, v in _rules.items()}
regexes_dict = {**regexes_dict, **inv_map}

output(random.choice(list(regexes_dict.items())))

('^(?=(STARZ).*)(?!.*(Arabia).*).*$|^(?=(STARZ).*)(?!.*(a^).*).*$|^(?=.*(STARZ).*)(?!.*(a^).*).*$|^(?=.*(STARZ).*)(?!.*(Your).*).*$|^(?=.*(STARZ).*)(?!.*(The '
 'Roku '
 'Channel).*).*$|^(?=(com.starz.starzplay.subscriptions).*)(?!.*(a^).*).*$|^(?=(^YouTube.+Starz).*)(?!.*(a^).*).*$|^(?=.*(Starz).*)(?!.*(a^).*).*$|^(?=(STARZ '
 '- Fake '
 'Cancel).*)(?!.*(a^).*).*$|^(?=(STARZ).*)(?!.*(a^).*).*$|^(?=.*(STARZ).*)(?!.*(The '
 'Roku Channel).*).*$|^(?=.*(STARZ).*)(?!.*(a^).*).*$|^(?=(STARZ - Fake '
 'New).*)(?!.*(a^).*).*$|^(?=(STARZ - Fake Cancel Passive '
 'Churn).*)(?!.*(a^).*).*$|^(?=(Your.*STARZ).*)(?!.*(a^).*).*$|^(?=(Your.*Starz).*)(?!.*((Orange)).*).*$|^(?=(Your.*STARZ).*)(?!.*(a^).*).*$|^(?=.*(STARZ).*)(?!.*(a^).*).*$|^(?=(STARZ '
 '- Fake New).*)(?!.*(a^).*).*$|^(?=(STARZ - Fake Cancel).*)(?!.*(a^).*).*$',
 8)


In [ ]:
"""
Build the dictionary to map signals from
description text -
The mapping dictionay of signal keywords 
can be configured in ROOT_DIR/config.py
"""

signals = {'^.*(' + ('|'.join(v)) + ').*$' : k for k, v in config.inputs['signals'].items()}

output(signals)

In [ ]:
"""
Build the dictionary to map signals from
description text -
The mapping dictionay of signal keywords 
can be configured in ROOT_DIR/config.py
"""


aservices = rules
aservices = aservices[aservices['service_name'].isin(config.inputs['allowed_services'])]
aservices = dict(zip(aservices['service_id'], aservices['service_name']))

output(aservices)

## Data Processing ##     

> Args:  
>  1. `regexes_dict (dict of str: int): Defines the regexes that map to service ids`

>  2. `signals (dict of str: int): Defines the regexes that map to signals from the description`

>  3. `aservices (dict of int: str): Defines the service ids that map to allowed service names`



In [ ]:
# Load the dataset
data = pd.read_csv(config.inputs['data']['fullpath'])

In [ ]:
"""
Requirement
There 3 types of statuses:
1. N (new): this transaction is new
2. U (update): this transaction is 
updated; discard the old one
3. D (delete): remove this transaction
"""

# Remove transactions with status 'D' as per requirements
data = data[data['status'] != 'D']

# Remove transactions that have an older entry
data = data.sort_values('last_updated').drop_duplicates('item_id',keep='last')

In [52]:
# Copy the description column 
data['service_id'] = data[['description']]

# Replace the description in the new service_id
# column with the matching service_id from
# regexes_dict that we created above
data['service_id'] = data[['service_id']].replace({'service_id':regexes_dict}, regex=True)

print(data.to_markdown())

print(data.to_html())

display(data)

|     |    item_id |    buyer_id | order_date   |   merchant_id | merchant_name   | status   | last_updated     | description                                                                                 |   service_id | service_name   | signal_type                                                  |
|----:|-----------:|------------:|:-------------|--------------:|:----------------|:---------|:-----------------|:--------------------------------------------------------------------------------------------|-------------:|:---------------|:-------------------------------------------------------------|
|   1 | 1191248937 | -1039890773 | 2016-02-19   |             6 | Netflix         | N        | 2016-02-20 11:18 | we've cancelled your Netflix Account. This change will be effective Sunday, March 20, 2016. |            3 | Netflix        | cancellation                                                 |
|   2 | 1221099193 |  -751620046 | 2016-02-27   |             6 | Netflix         | N      

,item_id,buyer_id,order_date,merchant_id,merchant_name,status,last_updated,description,service_id,service_name,signal_type
1,1191248937,-1039890773,2016-02-19,6,Netflix,N,2016-02-20 11:18,we've cancelled your Netflix Account. This cha...,3,Netflix,cancellation
2,1221099193,-751620046,2016-02-27,6,Netflix,N,2016-02-28 22:26,Thanks for joining Netflix!,3,Netflix,signup
3,1294618642,-78172962,2016-03-18,6,Netflix,N,2016-03-18 16:18,we've cancelled your Netflix Account. This cha...,3,Netflix,cancellation
5,1537007003,1799691237,2016-06-28,6,Netflix,N,2016-06-28 19:49,Thanks for joining Netflix!,3,Netflix,signup
8,1613595118,-959572143,2016-07-30,6,Netflix,N,2016-07-31 14:47,Thanks for coming back to Netflix!,3,Netflix,signup
...,...,...,...,...,...,...,...,...,...,...,...
165,9528512440,-1052896650,2016-07-31,6,Netflix,N,2020-10-08 09:51,we've cancelled your Netflix Account,3,Netflix,cancellation
166,9528512560,-1052896650,2016-08-08,6,Netflix,N,2020-10-08 09:51,Thanks for coming back to Netflix!,3,Netflix,signup
167,9548920339,1724361521,2016-11-15,6,Netflix,N,2020-10-12 09:41,Thanks for coming back to Netflix!,3,Netflix,signup
168,9707741036,209419893,2016-10-16,6,Netflix,N,2020-11-12 09:52,Thanks for coming back to Netflix!,3,Netflix,signup


In [ ]:
"""
Requirement
To simplify the project, you only need to 
consider the following services, although
the data may include many others.
● Netflix -> 3
● Hulu -> 1
● CBS All Access  -> 39
● Starz -> 8
● Showtime -> 12
"""
data = data[data['service_id'].isin(list(aservices.keys()))]

doutput(data.head())



# Add the service names corresponding 
# to the service id

# Copy the description column 

data['service_name'] = data[['service_id']]
data['service_name'] = data[['service_name']].replace({'service_name': aservices})


In [ ]:
"""
Requirement
You still need to figure out what kind 
of action (signup versus cancellation) 
the remaining transactions are about.
If there are trial signup and cancellation, 
please make your own judgment as to how 
to treat them.
"""
data['signal_type'] = data[['description']]
data['signal_type'] = data[['signal_type']].replace({'signal_type': signals}, regex = True)

In [ ]:
# Save the processed dataset locally
save_file(config.outputs['local']['fullpath'], data.to_csv())

**⭣ This is the local link to the processed file: ⭣**

In [ ]:
# Output Local URL
output(config.outputs['local']['fullpath'])

In [ ]:
"""
Save a copy of the processed dataset 
to Google. Use IAM roles for authentication.

Requirement
Output data is accessible from a common 
cloud storage (i.e. AWS S3 or Google Storage)
"""
from google.cloud import storage

client = storage.Client()
bucket = client.get_bucket(config.outputs['cloud']['bucket_name'])
blob = bucket.blob(config.outputs['local']['filename'])
blob.upload_from_filename(config.outputs['local']['fullpath'])


**⭣ This is the Cloud URL to the processed file: ⭣**

In [ ]:
# Output Cloud URL
output(blob.public_url)

In [ ]:
# print(data.to_string())